In [1]:
import requests
import json
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
import time 
import os
import json
import base64

In [2]:
url = 'https://www.cctd.com.cn/list-46-1.html'
div_class = 'new_list'
next_page_xpath = '//*[@id="pages"]/a[5]'
page_num = 2
keyword = "电煤价格"

In [42]:
def scrape_urls(url=None, div_class=None, keyword=None, urls_dict=None, page_num=1, next_page_xpath=None):
    if urls_dict is None:
        urls_dict = {}
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        for _ in range(page_num):
            if next_page_xpath is None and _ > 0:
                print("Warning: No next page XPath provided. Only scraping the first page.")
                break
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            if div_class is not None:
                url_div = soup.find_all('div', class_=div_class)
            else:
                url_div = soup.find_all('table')
            
            for div in url_div:
                for link in div.find_all('a'):
                    if keyword is None or (keyword is not None and keyword in link.text):
                        try:
                            urls_dict.update({link.text: link['href']})
                        except:
                            continue
        
            if next_page_xpath is not None:
                try:
                    next_page_element = wait.until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
                    next_page_element.click()
                except WebDriverException:
                    print("Warning: Next page button not clickable or not found. Stopping the scraping.")
                    break
        
    finally:
        driver.quit()

    return urls_dict

def store_urls_as_json(urls_dict, output_file):
    # Check if the output path is a full path
    if not os.path.isabs(output_file):
        # If it's not a full path, join it with the current working directory
        output_file = os.path.abspath(os.path.join(os.getcwd(), output_file))

    # Check if the output_file is a directory
    if os.path.isdir(output_file):
        # If it's a directory, create the full file path
        output_file = os.path.join(output_file, "urls_info.json")

    # Create the directory if it doesn't exist
    output_dir = os.path.dirname(output_file)
    os.makedirs(output_dir, exist_ok=True)

    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(urls_dict, file, ensure_ascii = False, indent=4)
        print(f"URLs stored as JSON in file: {output_file}")
    except IOError as e:
        print(f"Error occurred while writing to file: {output_file}")
        print(f"Error details: {str(e)}")
    except Exception as e:
        print("An error occurred during the file operation.")
        print(f"Error details: {str(e)}")

In [18]:
d1 = scrape_urls(url=url, div_class=div_class, keyword='指数', page_num=3, next_page_xpath=next_page_xpath)

In [44]:
len(d1)

54

In [39]:
os.path.realpath(os.path.join(os.getcwd(), '../data/raw/煤炭/'))

'C:\\Users\\Bill Chen\\Desktop\\UCSD Course\\MD Lab\\scrapeAll\\data\\raw\\煤炭'

In [43]:
store_urls_as_json(d1, "../data/raw/煤炭")

URLs stored as JSON in file: C:\Users\Bill Chen\Desktop\UCSD Course\MD Lab\scrapeAll\data\raw\煤炭\urls_info.json


In [ ]:
def scrape_urls(url = None, div_class = None, keyword = None, urls_dict = {}, page_num = 1):
    driver = webdriver.Chrome()
    driver.get(url)
    for _ in range(page_num):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        url_div = soup.find_all('div', class_ = div_class)
        for div in url_div:
            for link in div.find_all('a'):
            if keyword in link.text:
                try:
                    urls_dict.update({link.text: link['href']})
                except:
                    continue
    if driver.find_element(By.XPATH, next_page_xpath).is_enabled():
        driver.find_element(By.XPATH, next_page_xpath).click()
    return urls_dict

def store_urls_as_json(urls_dict, output_file):
    with open(output_file, 'w') as file:
        json.dump(urls_dict, file, indent=4)

# Example usage
url = 'https://example.com'
div_class = 'target-div-class'
keyword = 'example'
page_number = 3  # Scrape URLs from the first 3 pages
output_file = 'urls.json'

urls = scrape_urls(url, div_class, keyword, page_number)
store_urls_as_json(urls, output_file)


In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
url_div = soup.find('div', class_ = div_class)
for link in url_div.find_all('a'):
    print(link)

In [ ]:
def scrape_urls(url, div_class, keyword=None, page_number=None):
    urls = []
    current_page = 1

    while page_number is None or current_page <= page_number:
        response = requests.get(f'{url}?page={current_page}')
        soup = BeautifulSoup(response.content, 'html.parser')

        for div in soup.find_all('div', class_=div_class):
            for link in div.find_all('a'):
                url = link.get('href')
                name = link.text.strip()
                if keyword is None or keyword.lower() in name.lower():
                    urls.append({'name': name, 'url': url})

        current_page += 1

    return urls

def store_urls_as_json(urls, output_file):
    with open(output_file, 'w') as file:
        json.dump(urls, file, indent=4)

# Example usage
url = 'https://example.com'
div_class = 'target-div-class'
keyword = 'example'
page_number = 3  # Scrape URLs from the first 3 pages
output_file = 'urls.json'

urls = scrape_urls(url, div_class, keyword, page_number)
store_urls_as_json(urls, output_file)
